In [21]:
import numpy as np
import nibabel as nib
import os
import os.path as op
import sys

In [53]:
def copyfile(src, dst):
    for suffix in ['nii.gz', 'json']:
        if op.exists(f'{src}.{suffix}'):
            os.system(f'cp {src}.{suffix} {dst}.{suffix}')
        else:
            print(f'Warning: no file found {src}.{suffix}')


def createASLcontext(dst, type='pwi'):
    nii = nib.load(f'{dst}.nii.gz')
    ctx = dst[:-3] + 'aslcontext.tsv'
    with open(ctx, 'w') as f:
        if type == 'ct':
            for i in range(nii.shape[3]//2):
                f.write('control\nlabel\n')
        elif type == 'tc':
            for i in range(nii.shape[3]//2):
                f.write('label\ncontrol\n')
        elif type == 'diff':
            if len(nii.shape) == 3:
                f.write('deltam\n')
            else:
                for i in range(nii.shape[3]):
                    f.write('deltam\n')
        elif type == 'pwi':
            if len(nii.shape) == 3:
                f.write('cbf\n')
            else:
                for i in range(nii.shape[3]):
                    f.write('cbf\n')
        else:
            print(f'Error: unknown type {type}')
            return


def copyGE3D(src, dst):
    if op.exists(f'{src}.nii.gz') and op.exists(f'{src}.json'):
        if nib.load(f'{src}.nii.gz').get_fdata().shape[3] == 2:
            src_2vol = src
        else:
            src_8vol = src
    else:
        print(f'Warning: no file found {src}.nii.gz or {src}.json')
        return
    if op.exists(f'{src}a.nii.gz') and op.exists(f'{src}a.json'):
        if nib.load(f'{src}a.nii.gz').get_fdata().shape[3] == 2:
            src_2vol = src+'a'
        else:
            src_8vol = src+'a'
    else:
        print(f'Warning: no file found {src}a.nii.gz or {src}a.json')
        return
    os.system(f'fslroi {src_2vol}.nii.gz {dst}_run-01_asl.nii.gz 1 1')
    createASLcontext(dst+'_run-01_asl', type='pwi')
    os.system(f'fslroi {src_2vol}.nii.gz {dst}_run-01_m0scan.nii.gz 0 1')
    os.system(f'cp {src_2vol}.json {dst}_run-01_asl.json')
    os.system(f'cp {src_2vol}.json {dst}_run-01_m0scan.json')
    for i in range(2, 6):
        os.system(f'fslroi {src_8vol}.nii.gz {dst}_run-0{i}_asl.nii.gz {(i-2)*2} 1')
        createASLcontext(dst+f'_run-0{i}_asl', type='pwi')
        os.system(f'fslroi {src_8vol}.nii.gz {dst}_run-0{i}_m0scan.nii.gz {(i-2)*2+1} 1')
        os.system(f'cp {src_8vol}.json {dst}_run-0{i}_asl.json')
        os.system(f'cp {src_8vol}.json {dst}_run-0{i}_m0scan.json')


def copyGEeASL(src, dst):
    if op.exists(f'{src}.nii.gz') and op.exists(f'{src}.json'):
        # Add '\' before the parenthesis
        src = src.replace('(', '\(').replace(')', '\)')
        os.system(f'fslroi {src}.nii.gz {dst}_asl.nii.gz 0 7')
        createASLcontext(dst+'_asl', type='diff')
        os.system(f'fslroi {src}.nii.gz {dst}_pwi.nii.gz 7 1')
        os.system(f'fslroi {src}.nii.gz {dst}_m0scan.nii.gz 8 1')
        os.system(f'cp {src}.json {dst}_asl.json')
        os.system(f'cp {src}.json {dst}_pwi.json')
        os.system(f'cp {src}.json {dst}_m0scan.json')
    else:
        print(f'Warning: no file found {src}.nii.gz or {src}.json')

In [75]:
NIFTIDIR = '/Users/xinzhang/Downloads/mrc_asl_cic/data/NIFTI'
BIDSDIR = '/Users/xinzhang/Downloads/mrc_asl_cic/BIDS'
subno = 10

# anat image copy
for ses in range(1, 3):
    os.makedirs(op.join(BIDSDIR, f'sub-{subno:02d}/ses-{ses:02d}/anat'), exist_ok=True)
    src = op.join(NIFTIDIR, f'sub{subno:02d}/ing_s1', 'MPRAGE')
    dst = op.join(BIDSDIR, f'sub-{subno:02d}/ses-{ses:02d}/anat/sub-{subno:02d}_ses-{ses:02d}_acq-philips_T1w')
    copyfile(src, dst)
    src = op.join(NIFTIDIR, f'sub{subno:02d}/ing_s1', 'MPRAGE')
    dst = op.join(BIDSDIR, f'sub-{subno:02d}/ses-{ses:02d}/anat/sub-{subno:02d}_ses-{ses:02d}_acq-philips_T1w')
    copyfile(src, dst)
    src = op.join(NIFTIDIR, f'sub{subno:02d}/ge_s1', '3D_SAG_T1_MP-RAGE_TI800')
    dst = op.join(BIDSDIR, f'sub-{subno:02d}/ses-{ses:02d}/anat/sub-{subno:02d}_ses-{ses:02d}_acq-GE_T1w')
    copyfile(src, dst)
    src = op.join(NIFTIDIR, f'sub{subno:02d}/ge_s1', '3D_SAG_T1_MP-RAGE_TI800')
    dst = op.join(BIDSDIR, f'sub-{subno:02d}/ses-{ses:02d}/anat/sub-{subno:02d}_ses-{ses:02d}_acq-GE_T1w')
    copyfile(src, dst)

# perf image copy
for ses in range(1, 3):
    os.makedirs(op.join(BIDSDIR, f'sub-{subno:02d}/ses-{ses:02d}/perf'), exist_ok=True)
    src = op.join(NIFTIDIR, f'sub{subno:02d}/ing_s{ses:01d}', 'WIP_SOURCE_-_2dREST_PROD_pCASL-nonorm')
    dst = op.join(BIDSDIR, f'sub-{subno:02d}/ses-{ses:02d}/perf/sub-{subno:02d}_ses-{ses:02d}_task-rest_acq-philips2d_asl')
    copyfile(src, dst)
    createASLcontext(dst, type='ct')
    src = op.join(NIFTIDIR, f'sub{subno:02d}/ing_s{ses:01d}', '2dM0_PROD_pCASL')
    dst = op.join(BIDSDIR, f'sub-{subno:02d}/ses-{ses:02d}/perf/sub-{subno:02d}_ses-{ses:02d}_task-rest_acq-philips2d_m0scan')
    copyfile(src, dst)
    src = op.join(NIFTIDIR, f'sub{subno:02d}/ing_s{ses:01d}', 'WIP_SOURCE_-_2DACT_PROD_pCASL-nonorm')
    dst = op.join(BIDSDIR, f'sub-{subno:02d}/ses-{ses:02d}/perf/sub-{subno:02d}_ses-{ses:02d}_task-task_acq-philips2d_asl')
    copyfile(src, dst)
    createASLcontext(dst, type='ct')
    src = op.join(NIFTIDIR, f'sub{subno:02d}/ing_s{ses:01d}', '2dM0_PROD_pCASLa')
    dst = op.join(BIDSDIR, f'sub-{subno:02d}/ses-{ses:02d}/perf/sub-{subno:02d}_ses-{ses:02d}_task-task_acq-philips2d_m0scan')
    copyfile(src, dst)
    src = op.join(NIFTIDIR, f'sub{subno:02d}/ing_s{ses:01d}', 'WIP_SOURCE_-_REST_PROD_3D_pCASL_6mm_noNorm')
    dst = op.join(BIDSDIR, f'sub-{subno:02d}/ses-{ses:02d}/perf/sub-{subno:02d}_ses-{ses:02d}_task-rest_acq-philips3d_asl')
    copyfile(src, dst)
    createASLcontext(dst, type='ct')
    src = op.join(NIFTIDIR, f'sub{subno:02d}/ing_s{ses:01d}', 'WIP_SOURCE_-_Mo3d')
    dst = op.join(BIDSDIR, f'sub-{subno:02d}/ses-{ses:02d}/perf/sub-{subno:02d}_ses-{ses:02d}_task-rest_acq-philips3d_m0scan')
    copyfile(src, dst)
    src = op.join(NIFTIDIR, f'sub{subno:02d}/ing_s{ses:01d}', 'SOURCE_-_ACT_PROD_3D_pCASL_6mm_noNorm')
    dst = op.join(BIDSDIR, f'sub-{subno:02d}/ses-{ses:02d}/perf/sub-{subno:02d}_ses-{ses:02d}_task-task_acq-philips3d_asl')
    copyfile(src, dst)
    createASLcontext(dst, type='ct')
    src = op.join(NIFTIDIR, f'sub{subno:02d}/ing_s{ses:01d}', 'WIP_SOURCE_-_Mo3d')
    dst = op.join(BIDSDIR, f'sub-{subno:02d}/ses-{ses:02d}/perf/sub-{subno:02d}_ses-{ses:02d}_task-task_acq-philips3d_m0scan')
    copyfile(src, dst)

    src = op.join(NIFTIDIR, f'sub{subno:02d}/ge_s{ses:01d}', '3D_Ax_ASL_5_starts')
    dst = op.join(BIDSDIR, f'sub-{subno:02d}/ses-{ses:02d}/perf/sub-{subno:02d}_ses-{ses:02d}_task-rest_acq-GE3D')
    copyGE3D(src, dst)
    src = op.join(NIFTIDIR, f'sub{subno:02d}/ge_s{ses:01d}', '3D_Ax_ASL_ACT_5_starts')
    dst = op.join(BIDSDIR, f'sub-{subno:02d}/ses-{ses:02d}/perf/sub-{subno:02d}_ses-{ses:02d}_task-task_acq-GE3D')
    copyGE3D(src, dst)
    src = op.join(NIFTIDIR, f'sub{subno:02d}/ge_s{ses:01d}', 'NOT_DIAGNOSTIC_(Raw)_eASL_7_delays_real')
    dst = op.join(BIDSDIR, f'sub-{subno:02d}/ses-{ses:02d}/perf/sub-{subno:02d}_ses-{ses:02d}_task-rest_acq-GEeASL')
    copyGEeASL(src, dst)
    src = op.join(NIFTIDIR, f'sub{subno:02d}/ge_s{ses:01d}', 'NOT_DIAGNOSTIC_(Raw)_eASL_7_delays_ACT_real')
    dst = op.join(BIDSDIR, f'sub-{subno:02d}/ses-{ses:02d}/perf/sub-{subno:02d}_ses-{ses:02d}_task-task_acq-GEeASL')
    copyGEeASL(src, dst)

# fmap image copy
for ses in range(1, 3):
    os.makedirs(op.join(BIDSDIR, f'sub-{subno:02d}/ses-{ses:02d}/fmap'), exist_ok=True)
    src = op.join(NIFTIDIR, f'sub{subno:02d}/ing_s{ses:01d}', 'B0_e1a')
    dst = op.join(BIDSDIR, f'sub-{subno:02d}/ses-{ses:02d}/fmap/sub-{subno:02d}_ses-{ses:02d}_acq-philips_magnitude')
    copyfile(src, dst)
    src = op.join(NIFTIDIR, f'sub{subno:02d}/ing_s{ses:01d}', 'B0_e2_fieldmaphza')
    dst = op.join(BIDSDIR, f'sub-{subno:02d}/ses-{ses:02d}/fmap/sub-{subno:02d}_ses-{ses:02d}_acq-philips_fieldmap')
    copyfile(src, dst)

print('Done')

Done


In [67]:
import bids
layout = bids.BIDSLayout(BIDSDIR, validate=True)
layout.get()

[<BIDSJSONFile filename='/Users/xinzhang/Downloads/mrc_asl_cic/BIDS/dataset_description.json'>,
 <BIDSJSONFile filename='/Users/xinzhang/Downloads/mrc_asl_cic/BIDS/sub-01/ses-01/anat/sub-01_ses-01_acq-GE_T1w.json'>,
 <BIDSImageFile filename='/Users/xinzhang/Downloads/mrc_asl_cic/BIDS/sub-01/ses-01/anat/sub-01_ses-01_acq-GE_T1w.nii.gz'>,
 <BIDSJSONFile filename='/Users/xinzhang/Downloads/mrc_asl_cic/BIDS/sub-01/ses-01/anat/sub-01_ses-01_acq-philips_T1w.json'>,
 <BIDSImageFile filename='/Users/xinzhang/Downloads/mrc_asl_cic/BIDS/sub-01/ses-01/anat/sub-01_ses-01_acq-philips_T1w.nii.gz'>,
 <BIDSJSONFile filename='/Users/xinzhang/Downloads/mrc_asl_cic/BIDS/sub-01/ses-01/fmap/sub-01_ses-01_acq-philips_fieldmap.json'>,
 <BIDSImageFile filename='/Users/xinzhang/Downloads/mrc_asl_cic/BIDS/sub-01/ses-01/fmap/sub-01_ses-01_acq-philips_fieldmap.nii.gz'>,
 <BIDSJSONFile filename='/Users/xinzhang/Downloads/mrc_asl_cic/BIDS/sub-01/ses-01/fmap/sub-01_ses-01_acq-philips_magnitude.json'>,
 <BIDSImageFil